# Initial Imports

In [1]:
from bigdata_a3_utils import *
import shutil
import os
from pathlib import Path
import logging
import tarfile
import zipfile

In [2]:
cache_dir = get_cache_directory()

[INFO] Current cache directory: C:\Users\saeed\.cache\huggingface\datasets
[NOTE] To use a custom cache directory, set HF_DATASETS_CACHE before importing datasets.
Example:
    import os
    os.environ['HF_DATASETS_CACHE'] = 'C:\\your\\custom\\path'
    from datasets import load_dataset



In [3]:
default_cache_dir = default_cache_path()

[INFO] Your default cache path: "C:\Users\saeed\.cache\huggingface\datasets"


# Checking availble disk space
- Files could also be stored on external drive
- Simply change file path in save_path variable

In [4]:
# Get the disk where your cache directory is located
cache_path = cache_dir
drive = os.path.splitdrive(cache_path)[0]

# Get disk usage statistics
total, used, free = shutil.disk_usage(drive)

# Convert to GB for easier reading
total_gb = total / (1024**3)
used_gb = used / (1024**3)
free_gb = free / (1024**3)

print(f"Drive {drive}:")
print(f"Total: {total_gb:.2f} GB")
print(f"Used: {used_gb:.2f} GB")
print(f"Free: {free_gb:.2f} GB")

# Check if you have enough space (e.g., 220GB to be safe)
required_space = 220  # GB
if free_gb > required_space:
    print(f"✅ You have enough space for the 200GB dataset (with some buffer)")
else:
    print(f"❌ Not enough space! You need at least {required_space} GB, but only have {free_gb:.2f} GB free")
    print(f"Consider setting a custom cache directory on a drive with more space")

Drive C::
Total: 474.72 GB
Used: 432.53 GB
Free: 42.19 GB
❌ Not enough space! You need at least 220 GB, but only have 42.19 GB free
Consider setting a custom cache directory on a drive with more space


# Division of categories into subsets per person:


In [5]:
subset_sizes = [11, 11, 12]  # Define the sizes of each subset

# Create subsets
subsets = []
start_index = 0
for size in subset_sizes:
    subsets.append(VALID_CATEGORIES[start_index:start_index + size])
    start_index += size

# Unpack the subsets into individual variables
kailash_subset, saeed_subset1, johnny_subset2 = subsets

print(f"Subset list: {subsets}")
print(f"Length: {len(VALID_CATEGORIES)}")
# Output the subsets


Subset list: [['All_Beauty', 'Amazon_Fashion', 'Appliances', 'Arts_Crafts_and_Sewing', 'Automotive', 'Baby_Products', 'Beauty_and_Personal_Care', 'Books', 'CDs_and_Vinyl', 'Cell_Phones_and_Accessories', 'Clothing_Shoes_and_Jewelry'], ['Digital_Music', 'Electronics', 'Gift_Cards', 'Grocery_and_Gourmet_Food', 'Handmade_Products', 'Health_and_Household', 'Health_and_Personal_Care', 'Home_and_Kitchen', 'Industrial_and_Scientific', 'Kindle_Store', 'Magazine_Subscriptions'], ['Movies_and_TV', 'Musical_Instruments', 'Office_Products', 'Patio_Lawn_and_Garden', 'Pet_Supplies', 'Software', 'Sports_and_Outdoors', 'Subscription_Boxes', 'Tools_and_Home_Improvement', 'Toys_and_Games', 'Video_Games', 'Unknown']]
Length: 34


# Setting up log directory and logging system
- Used for proof of file acquisition

In [6]:
log_dir = Path("logs")
log_dir.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    filename=log_dir / 'download_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Error handling:
- In the event of an issue where the folder was not dowloaded properly
- In the event of a issue with preprocessing any particular category (OS error, Permissions error, Temp file being deleted before full extraction) it is likely the initial download was corrupted by some means. As such simply redownload via the last cell, again specifying the failed category

In [7]:
def verify_compressed_file(file_path):
    """
    Verifies if a compressed file (tar.gz, zip, etc.) is valid and not corrupted.
    
    Args:
        file_path: Path to the compressed file
        
    Returns:
        bool: True if the file is valid, False otherwise
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"Error: File does not exist: {file_path}")
        return False
        
    try:
        if str(file_path).endswith('.tar.gz') or str(file_path).endswith('.tgz'):
            # Test if tar.gz file is valid
            with tarfile.open(file_path, 'r:gz') as tar:
                # Just check the integrity by attempting to list contents
                tar.getnames()
                print(f"✓ Verified {file_path} is a valid tar.gz file")
                return True
                
        else:
            print(f"Error: Unsupported file format for {file_path}")
            return False
            
    except Exception as e:
        print(f"Error verifying {file_path}: {str(e)}")
        return False

# Downloading the folders and metadata
- Just change the file path in save_path accordingly to where you want it stored
- VALID_CATEGORIES will download all metadata and review data for all 34 categories, otherwise choose a subset from above to work with. 

In [11]:
save_path = Path(r"D:\BigData")
save_path.mkdir(parents=True, exist_ok=True)

for category in VALID_CATEGORIES:
    logging.info(f"n------Processing {category}------")
    print(f"\n------Processing {category}------")

    download_all_amazon_reviews(base_save_path=save_path, 
                                categories=[category], compress=False)

    review_folder = save_path / f"raw_review_{category}"
    metadata_folder = save_path / f"raw_meta_{category}"

    if review_folder.exists():
        compressed = compress_folder(review_folder, 
                                        compression_format="gz", 
                                        level=6)
        
        logging.info(f"Compressed reviews to: {compressed}")
        print(f"Compressed reviews to: {compressed}")

    if metadata_folder.exists():
        compressed = compress_folder(metadata_folder, 
                                        compression_format="gz", 
                                        level=6)
        
        logging.info(f"Compressed metadata to: {compressed}")
        print(f"Compressed metadata to: {compressed}")

    
    verify_compressed_file(review_folder)
    verify_compressed_file(metadata_folder) 


------Processing All_Beauty------
⠋ Processing All_Beauty 

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

⠹ Processing All_Beauty 

c:\Users\saeed\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saeed\.cache\huggingface\hub\datasets--McAuley-Lab--Amazon-Reviews-2023. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


⠦ Processing All_Beauty 

...

Saving the dataset (0/2 shards):   0%|          | 0/213593 [00:00<?, ? examples/s]

[DONE] raw_meta_Musical_Instruments downloaded
✅ Processing Musical_Instruments 

🎉 Download summary:
  - Successfully processed: 1/1 categories
Compressed reviews to: D:\BigData\raw_review_Musical_Instruments.tar.gz
Compressed metadata to: D:\BigData\raw_meta_Musical_Instruments.tar.gz
Error: File does not exist: D:\BigData\raw_review_Musical_Instruments
Error: File does not exist: D:\BigData\raw_meta_Musical_Instruments

------Processing Office_Products------
⠸ Processing Office_Products 

c:\Users\saeed\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:752: UserWarning: Not enough free disk space to download the file. The expected file size is: 5776.79 MB. The target location C:\Users\saeed\.cache\huggingface\hub\datasets--McAuley-Lab--Amazon-Reviews-2023\blobs only has 3576.70 MB free disk space.
  warnings.warn(


Office_Products.jsonl:   0%|          | 0.00/5.78G [00:00<?, ?B/s]

💥 Processing Office_Products
Failed to process category 'Office_Products': [Errno 28] No space left on device

🎉 Download summary:
  - Successfully processed: 0/1 categories
  - Failed: 1/1 categories
    - Office_Products: [Errno 28] No space left on device
Error: File does not exist: D:\BigData\raw_review_Office_Products
Error: File does not exist: D:\BigData\raw_meta_Office_Products

------Processing Patio_Lawn_and_Garden------
💥 Processing Patio_Lawn_and_Garden
Failed to process category 'Patio_Lawn_and_Garden': Not enough disk space. Needed: Unknown size (download: Unknown size, generated: Unknown size, post-processed: Unknown size)

🎉 Download summary:
  - Successfully processed: 0/1 categories
  - Failed: 1/1 categories
    - Patio_Lawn_and_Garden: Not enough disk space. Needed: Unknown size (download: Unknown size, generated: Unknown size, post-processed: Unknown size)
Error: File does not exist: D:\BigData\raw_review_Patio_Lawn_and_Garden
Error: File does not exist: D:\BigData

# Note
- the cell below is the one you would use to redownload any specific category that failed during initial download or if OS error, permissions error, file locking error

In [ ]:
save_path = Path(r"D:\BigData")
save_path.mkdir(parents=True, exist_ok=True)
# category = ""
remaining= ["Electronics", "Kindle_Store", "Movies_and_TV", 
            "Sports_and_Outdoors", "Unknown"
            ]

for category in remaining:
    logging.info(f"n------Processing {category}------")
    print(f"\n------Processing {category}------")

    download_all_amazon_reviews(base_save_path=save_path, 
                                categories=[category], compress=False)

    review_folder = save_path / f"raw_review_{category}"
    metadata_folder = save_path / f"raw_meta_{category}"

    if review_folder.exists():
        compressed = compress_folder(review_folder, 
                                        compression_format="gz", 
                                        level=6)
        
        logging.info(f"Compressed reviews to: {compressed}")
        print(f"Compressed reviews to: {compressed}")

    if metadata_folder.exists():
        compressed = compress_folder(metadata_folder, 
                                        compression_format="gz", 
                                        level=6)
        
        logging.info(f"Compressed metadata to: {compressed}")
        print(f"Compressed metadata to: {compressed}")

    verify_compressed_file(review_folder)
    verify_compressed_file(metadata_folder) 


------Processing Beauty_and_Personal_Care------
[SKIP] raw_review_Beauty_and_Personal_Care already exists
⠧ Processing Beauty_and_Personal_Care 

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

⠏ Processing Beauty_and_Personal_Care 

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

⠹ Processing Beauty_and_Personal_Care 

...

⠦ Processing Unknown 

Loading dataset shards:   0%|          | 0/43 [00:00<?, ?it/s]

⠸ Processing Unknown 

Saving the dataset (0/43 shards):   0%|          | 0/63814110 [00:00<?, ? examples/s]

[DONE] raw_review_Unknown downloaded
💥 Processing Unknown
Failed to process category 'Unknown': (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: df372fc8-bad9-4e85-8f54-e60203ad1539)')

🎉 Download summary:
  - Successfully processed: 0/1 categories
  - Failed: 1/1 categories
    - Unknown: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: df372fc8-bad9-4e85-8f54-e60203ad1539)')
Compressed reviews to: D:\BigData\raw_review_Unknown.tar.gz
Error: File does not exist: D:\BigData\raw_review_Unknown
Error: File does not exist: D:\BigData\raw_meta_Unknown
